<a href="https://colab.research.google.com/github/qu1r0ra/philippine-machine-translation/blob/chore%2Fpolish-files/notebooks/02b_modeling_nmt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modeling in Neural Machine Translation (NMT)

## Notes

The author who trained the Transformer model did not have sufficient computational resources to train locally, so he decided to train it via Google Colab.

Unfortunately, the free plan in Colab also did not suffice for training as the free GPUs have rate limits, so the author also decided to purchase compute units to access said GPUs. You may also need Colab compute units to replicate the results of this notebook.

## Environment Setup

First, let us uninstall Colab's preinstalled `torch`, `torchvision`, and `torchaudio` packages.

We need to downgrade them as the `OpenNMT-py` version that the authors used for training requires older versions.

**NOTE:** You only need to run this the first time.

In [ ]:
!pip uninstall torch torchvision torchaudio -y

Found existing installation: torch 2.8.0+cu126
Uninstalling torch-2.8.0+cu126:
  Successfully uninstalled torch-2.8.0+cu126
Found existing installation: torchvision 0.23.0+cu126
Uninstalling torchvision-0.23.0+cu126:
  Successfully uninstalled torchvision-0.23.0+cu126
Found existing installation: torchaudio 2.8.0+cu126
Uninstalling torchaudio-2.8.0+cu126:
  Successfully uninstalled torchaudio-2.8.0+cu126


Next, let us install `condacolab` so we can install the older package versions via `conda`.

The authors tried using `pip` to install the older versions but they were somehow unlisted and thus could not be installed.

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:08
🔁 Restarting kernel...


Now we can install the older versions required by our `OpenNMT-py` version.

In [ ]:
!conda install pytorch==2.1.1 torchvision==0.16.1 torchaudio==2.1.1 \
  pytorch-cuda=11.8 -c pytorch -c nvidia

Channels:
 - pytorch
 - nvidia
 - conda-forge
Platform: linux-64
Solving environment: - \ | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - pytorch-cuda=11.8
    - pytorch==2.1.1
    - torchaudio==2.1.1
    - torchvision==0.16.1


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _openmp_mutex-4.5          |       5_kmp_llvm           8 KB  conda-forge
    aom-3.5.0                  |       h27087fc_0         2.7 MB  conda-forge
    blas-2.116                 |              mkl          13 KB  conda-forge
    blas-devel-3.9.0           |   16_linux64_mkl          12 KB  conda-forge
    ca-certificates-2025.10.5  |       hbd8a1cb_0         152 KB  conda-forge
    certifi-2025.10.5          |     pyhd8ed1ab_0         156 KB  conda-forge
    conda-24.11.3              |  py311h38be061_0         1.1 MB  conda-forge
    cpython-3.11.14     

In [ ]:
!pip install OpenNMT-py==3.4.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.0/37.0 MB 150.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 162.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 125.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 168.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 124.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.3/32.3 MB 158.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 191.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 842.9/842.9 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 115.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 132.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 140.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 1

Lastly, let us create the folders where we will be uploading our data.

In [ ]:
!mkdir -p data outputs

## Data Upload

Next, we need to upload the necessary files to train our model.

At this point, it is assumed that:
1. You have cloned the GitHub repository https://github.com/qu1r0ra/philippine-machine-translation.
2. You have gone through `00_setup.ipynb` and `01b_preprocessing_nmt.ipynb` in the repository.

To prepare the data needed for this notebook, simply follow these instructions:
1. Upload `config.yaml` from the repository to the Colab filesystem (`/content`). You know you are in the right path if you can also see the file `condacolab_install.log`.
3. Upload `train.src`, `train.tgt`, `valid.src`, and `valid.tgt` from `data/processed/` in the repository to the `data/` folder in Colab.

Let us check the first few lines of our training and validation data.

In [ ]:
!head -n 5 data/gru-aug-cbk/train.src

busa nanghagdaw siya didto sa uma hangtod sa pagkahapon ug iyang gigiok ang iyang hinagdaw ug kini mga usa ka epha sa sebada
sukad sa mga adlaw sa among mga katigulangan hangtod niining adlawa kami hilabihan ka makasasala ug tungod sa among mga kalapasan kami ang among mga hari ug ang among mga pari gitugyan ngadto sa kamot sa mga hari sa kayutaan ngadto sa espada ngadto sa pagkabinihag ug sa pagkainilogan ug ngadto sa kaulawan hangtod niining adlawa
unya ibutang nila sa ibabaw niini ang tanang mga kahimanan sa halaran nga gigamit sa pagalagad diha niini ang mga sanggaan sa baga ang mga tinidor ug ang mga pala ug ang mga planggana ang tanang mga galamiton sa halaran ug tabonan nila kini sa usa ka tabon nga panit sa kanding ug itaod ang mga yayongan niini
sa diha nga mabuhat ang hustisya kalipay kini sa matarong apan kapukanan sa tigbuhat ug daotan
ang mga anak nga lalaki ni ham si cus si ehipto si put ug si canaan


In [ ]:
!head -n 5 data/gru-aug-cbk/train.tgt

espigó pues en el campo hasta la noche y cuando desgranó lo que había recogido era como un efa de cebada
desde los días de nuestros padres hasta este día hemos vivido en gran pecado y por nuestras iniquidades nosotros nuestros reyes y nuestros sacerdotes hemos sido entregados en manos de los reyes de los países a la espada al cautiverio al robo y a la vergüenza que cubre nuestro rostro como todavía sucede
pondrán sobre él todos los instrumentos que se emplean en su servicio las paletas los garfios los braseros y los tazones todos los utensilios del altar extenderán sobre él la cubierta de pieles de tejones y le pondrán además las varas
alegría es para el justo practicar la justicia pero un desastre para los que cometen iniquidad
los hijos de cam cus mizraim fut y canaán


In [ ]:
!head -n 5 data/gru-aug-cbk/valid.src

kining unom ka siyudad mahimong dalangpanan alang sa katawhan sa israel ug alang sa mga dumuduong ug alang sa mga langyaw aron nga si bisan kinsa nga makapatay ug tawo nga wala tuyoa makadangop didto
ta ama gayot came con ustedes y hende lang el buen noticia ta dale came con ustedes pero ta otorga came dale masquin pa el di amon vida cay tiene gayot came grande amor para con ustedes
ug si safan nga sekretaryo miadto sa hari ug gisuginlan pagusab ang hari gikuha sa imong mga sulugoon ang salapi nga nakita didto sa balay ug gihatag sa kamot sa mga mamumuo nga nagbantay sa balay sa ginoo
ug sila makigharong kanimo uban ang pagdumot ug kuhaon ang tanang bunga sa imong paghago ug biyaan ikaw nga hubo ug walay bisti ug ang pagkahubo sa imong pagkamakihilawason madayag ang imong kalaway ug ang imong pagkamakihilawason
unya gidala ni samuel si saul ug ang iyang sulugoon ngadto sa kananan ug gipalingkod sila sa kinaibabwan nga dapit uban kanila nga mga dinapit nga mga ka tawo


In [ ]:
!head -n 5 data/gru-aug-cbk/valid.tgt

estas seis ciudades serán de refugio para los hijos de israel para el extranjero y el que habite entre ellos para que huya allá cualquiera que hiera de muerte a otro sin intención
tan grande es nuestro afecto por vosotros que hubiéramos querido entregaros no solo el evangelio de dios sino también nuestras propias vidas porque habéis llegado a sernos muy queridos
luego el escriba safán se presentó ante el rey y le rindió cuentas diciendo tus siervos han recogido el dinero que se halló en el templo y se lo han entregado a los que hacen la obra los que tienen a su cargo el arreglo de la casa de jehová
los cuales procederán contigo con odio y tomarán todo el fruto de tu labor te dejarán desnuda por completo y se descubrirá la inmundicia de tus fornicaciones tu lujuria y tu prostitución
entonces samuel tomó a saúl y a su criado los introdujo a la sala y les dio un lugar a la cabecera de los convidados que eran unos treinta hombres


(will soon write stuff about the data)

We also need to install NumPy with version `<2.0` for `OpenNMT-py`.

In [ ]:
!pip install "numpy<2.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 197.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.4
    Uninstalling numpy-2.3.4:
      Successfully uninstalled numpy-2.3.4


## Training a Gated Recurrent Unit (GRU) with OpenNMT-py

The advantage with using `OpenNMT-py` is that we only need to configure the parameters and hyperparameters of the model with a `config.yaml` file. After doing so, the training can be ran with one line.

Normally, we would need to code the architecture and mechanisms of the model ourselves in deep learning frameworks like `PyTorch` and `TensorFlow`. As an aside, `OpenNMT` has implementations in both of these frameworks, but the authors chose the PyTorch implementation, thus named `OpenNMT-py`.

P.S.:
- In retrospect, the author shouldn't have went with this...

In [ ]:
!onmt_build_vocab -config config-gru-aug.yaml

In [ ]:
!onmt_train -config config-gru-aug.yaml

This is literally it. Now we just have to wait until the model finishes training.

Do note that training even a small GRU like may take hours, depending on various factors such as:
- the size of your dataset
- the parameters you set in your `config.yaml`
- the power of your GPU
- etc.

## Training the Same Exact Model Architecture on Non-Augmented Data (i.e., no Noise Injection)

In [ ]:
!onmt_build_vocab -config config-gru-base.yaml

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2025-11-08 20:24:31,903 INFO] Counter vocab from 5000 samples.
[2025-11-08 20:24:31,903 INFO] Build vocab on 5000 transformed examples/corpus.
[2025-11-08 20:24:32,111 INFO] Counters src: 8180
[2025-11-08 20:24:32,112 INFO] Counters tgt: 10994


In [ ]:
!onmt_train -config config-gru-base.yaml

[2025-11-08 20:24:45,510 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2025-11-08 20:24:45,510 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2025-11-08 20:24:45,510 INFO] Missing transforms field for valid data, set to default: [].
[2025-11-08 20:24:45,510 INFO] Parsed 2 corpora from -data.
[2025-11-08 20:24:45,510 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
[2025-11-08 20:24:45,539 INFO] The first 10 tokens of the vocabs are:['<unk>', '<blank>', '<s>', '</s>', 'sa', 'ug', 'ang', 'nga', 'mga', 'iyang']
[2025-11-08 20:24:45,539 INFO] The decoder start token is: <s>
[2025-11-08 20:24:45,539 INFO] Building model...
[2025-11-08 20:24:45,811 INFO] Switching model to float32 for amp/apex_amp
[2025-11-08 20:24:45,812 INFO] Non quantized layer compute is fp32
[2025-11-08 20:24:45,966 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Eleme

## Training the Same Exact Model Architecture on Data Augmented with Chavacano-Spanish

In [ ]:
!onmt_build_vocab -config config-gru-aug-cbk.yaml

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2025-11-08 15:40:37,574 INFO] Counter vocab from 5000 samples.
[2025-11-08 15:40:37,574 INFO] Build vocab on 5000 transformed examples/corpus.
[2025-11-08 15:40:37,768 INFO] Counters src: 8936
[2025-11-08 15:40:37,768 INFO] Counters tgt: 10839


In [ ]:
!onmt_train -config config-gru-aug-cbk.yaml

[2025-11-08 15:40:49,645 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2025-11-08 15:40:49,645 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2025-11-08 15:40:49,645 INFO] Missing transforms field for valid data, set to default: [].
[2025-11-08 15:40:49,645 INFO] Parsed 2 corpora from -data.
[2025-11-08 15:40:49,645 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
[2025-11-08 15:40:49,673 INFO] The first 10 tokens of the vocabs are:['<unk>', '<blank>', '<s>', '</s>', 'sa', 'ug', 'ang', 'nga', 'mga', 'si']
[2025-11-08 15:40:49,673 INFO] The decoder start token is: <s>
[2025-11-08 15:40:49,673 INFO] Building model...
[2025-11-08 15:40:49,938 INFO] Switching model to float32 for amp/apex_amp
[2025-11-08 15:40:49,938 INFO] Non quantized layer compute is fp32
[2025-11-08 15:40:50,216 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementw

# Translation

Let's perform translation here too since it needs `OpenNMT-py`.

Make sure you have the ff. uploaded:
- testing data (i.e., `test.src`) in `data/`
- models in `outputs/`

In [ ]:
# Common paths
SRC = "data/test.src"

# Config files
CONFIG_BASE = "config-gru-base.yaml"
CONFIG_AUG = "config-gru-aug.yaml"
CONFIG_AUG_CBK = "config-gru-aug-cbk.yaml"
CONFIG_TRANSLATE = "translate.yaml"

# Model checkpoints
MODEL_BASE = "/content/drive/MyDrive/opennmt_models/gru-base/model_gru_base_step_4000.pt"
MODEL_AUG = "/content/drive/MyDrive/opennmt_models/gru-aug/model_gru_aug_step_28500.pt"
MODEL_AUG_CBK = "/content/drive/MyDrive/opennmt_models/gru-aug-cbk/model_gru_aug_cbk_step_5750.pt"

# Translation outputs
OUT_BASE="outputs/trans-base.txt"
OUT_AUG="outputs/trans-aug.txt"
OUT_AUG_CBK="outputs/trans-aug-cbk.txt"

In [ ]:
# Translate with the base model
!onmt_translate \
    -model $MODEL_BASE \
    -src $SRC \
    -output $OUT_BASE \
    -gpu 0 \
    -batch_size 64 \
    -beam_size 5

# # Translate with the augmented model
# !onmt_translate \
#     -model $MODEL_AUG \
#     -src $SRC \
#     -output $OUT_AUG \
#     -gpu 0 \
#     -batch_size 64 \
#     -beam_size 5

# # Translate with the CBK-augmented model
# !onmt_translate \
#     -model $MODEL_AUG_CBK \
#     -src $SRC \
#     -output $OUT_AUG_CBK \
#     -gpu 0 \
#     -batch_size 64 \
#     -beam_size 5

[2025-11-08 20:47:29,999 INFO] Loading checkpoint from /content/drive/MyDrive/opennmt_models/gru-base/model_gru_base_step_4000.pt
[2025-11-08 20:47:30,387 INFO] Loading data into the model
Traceback (most recent call last):
  File "/usr/local/bin/onmt_translate", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/site-packages/onmt/bin/translate.py", line 67, in main
    translate(opt)
  File "/usr/local/lib/python3.11/site-packages/onmt/bin/translate.py", line 37, in translate
    _, _ = translator._translate(
           ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/onmt/translate/translator.py", line 510, in _translate
    translations = xlation_builder.from_batch(batch_data)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/onmt/translate/translation.py", line 98, in from_batch
    pred_sents = [
                 ^
  File "/usr/local/lib/python3.11/site-packag

The author doesn't know why, but he's experiencing several problems trying to get this to run. After two hours of no progress, he chose to implement the architecture through `PyTorch`.

It's painful, but that's life.

Lastly, mount your Google Drive and save the models you trained along with their translations.

P.S.:
- No need for this unless you want to keep the models you ended up training.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir -p /content/drive/MyDrive/opennmt_models
!cp -r outputs /content/drive/MyDrive/opennmt_models/